In [3]:
import os
os.environ['DISPLAY'] = ':1'
import open3d as o3d
import numpy as np
from typing import Literal
import copy
import pymeshlab

In [105]:
mesh_cfgs = [
    {
        'size': [0.250000, 0.150000, 0.010000],
        'translation': [0.000235, -0.000001, -0.175002],
        'rotation': [0.000024, 3.141593, -1.570796],
    },
    {
        'size': [0.250000, 0.350000, 0.010000],
        'translation': [-0.074754, 0.000001, 0.000002],
        'rotation': [-1.570804, 3.141593, -1.570796],
    },
    {
        'size': [0.150000, 0.350000, 0.010000],
        'translation': [0.000254, 0.124999, 0.000005],
        'rotation': [-1.570796, 3.141585, 0.000017],
    },
    {
        'size': [0.150000, 0.350000, 0.010000],
        'translation': [0.000246, -0.124999, 0.000002],
        'rotation': [-1.570796, 3.141585, 0.000026],
    },
    {
        'size': [0.250000, 0.150000, 0.010000],
        'translation': [0.000245, 0.000001, 0.175002],
        'rotation': [0.000018, 3.141593, -1.570796],
    },
    {
        'size': [0.268000, 0.350000, 0.010000],
        'translation': [0.126009, 0.038258, -0.000070],
        'rotation': [1.570772, 3.141586, -2.879811],
    }
]

meshes = []
for cfg in mesh_cfgs:
    size = cfg['size']
    mesh = o3d.geometry.TriangleMesh.create_box(size[0], size[1], size[2])
    mesh.translate(np.array([-size[0] / 2, -size[1] / 2, -size[2] / 2]))
    mesh_r = copy.deepcopy(mesh)
    r = cfg['rotation']
    mesh_r.rotate(np.linalg.inv(mesh.get_rotation_matrix_from_xyz([r[0], r[1], r[2]])), center=(0, 0, 0))
    # mesh_r.rotate(euler_to_rotation_matrix([r[0], r[1], r[2]]), center=(0, 0, 0))
    mesh = mesh_r
    t = cfg['translation']
    mesh.translate([t[0], t[1], t[2]])
    mesh.compute_vertex_normals()
    meshes.append(mesh)

mesh_coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2)
o3d.visualization.draw_geometries(meshes + [mesh_coord])

In [1]:
import os
os.environ['DISPLAY'] = ':1'
import xml.etree.ElementTree as ET
import numpy as np
import open3d as o3d


def parse_urdf(file_path):
    """Parses the URDF file and extracts box geometries with transformations."""
    tree = ET.parse(file_path)
    root = tree.getroot()

    boxes = []

    for link in root.findall("link"):
        for visual in link.findall("visual"):
            origin = visual.find("origin")
            geometry = visual.find("geometry")
            box = geometry.find("box") if geometry is not None else None

            if origin is not None and box is not None:
                xyz = np.array([float(v) for v in origin.get("xyz").split()])
                rpy = np.array([float(v) for v in origin.get("rpy").split()])
                size = np.array([float(v) for v in box.get("size").split()])

                boxes.append((xyz, rpy, size))

    return boxes


def create_box(size, xyz, rpy):
    """Creates an Open3D box mesh with the given size and transformation."""
    box = o3d.geometry.TriangleMesh.create_box(width=size[0], height=size[1], depth=size[2])
    box.translate([-size[0]/2, -size[1]/2, -size[2]/2])
    box.compute_vertex_normals()

    # Apply transformation
    transform = np.eye(4)

    # Rotation from RPY (Roll, Pitch, Yaw)
    R = euler_to_rotation_matrix(rpy)
    transform[:3, :3] = R
    transform[:3, 3] = xyz

    box.transform(transform)

    return box


def euler_to_rotation_matrix(rpy):
    """Converts roll, pitch, yaw (RPY) to a 3x3 rotation matrix."""
    roll, pitch, yaw = rpy

    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll), np.cos(roll)]
    ])

    Ry = np.array([
        [np.cos(pitch), 0, np.sin(pitch)],
        [0, 1, 0],
        [-np.sin(pitch), 0, np.cos(pitch)]
    ])

    Rz = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])

    return Rz @ Ry @ Rx


def visualize_urdf(boxes):
    """Visualizes the parsed URDF boxes using Open3D."""
    o3d_boxes = [create_box(size, xyz, rpy) for xyz, rpy, size in boxes]

    # Set uniform color for visibility
    for box in o3d_boxes:
        box.paint_uniform_color([0.7, 0.7, 0.7])

    o3d.visualization.draw_geometries(o3d_boxes)


urdf_file = "data_rlbench/urdf/close_box/box_base.urdf"  # Change to your URDF file path
boxes = parse_urdf(urdf_file)
visualize_urdf(boxes)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import os
os.environ['DISPLAY'] = ':1'
import xml.etree.ElementTree as ET
import numpy as np
import open3d as o3d


def euler_to_rotation_matrix(rpy):
    """Converts roll, pitch, yaw (RPY) angles to a 3x3 rotation matrix."""
    roll, pitch, yaw = rpy
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll), np.cos(roll)]
    ])
    Ry = np.array([
        [np.cos(pitch), 0, np.sin(pitch)],
        [0, 1, 0],
        [-np.sin(pitch), 0, np.cos(pitch)]
    ])
    Rz = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])
    return Rz @ Ry @ Rx


def rotation_matrix_to_euler(R):
    """Converts a 3x3 rotation matrix to roll, pitch, yaw angles.

    This uses a standard approach and is valid for non-singular configurations.
    """
    sy = np.sqrt(R[0, 0]**2 + R[1, 0]**2)
    singular = sy < 1e-6
    if not singular:
        roll  = np.arctan2(R[2, 1], R[2, 2])
        pitch = np.arctan2(-R[2, 0], sy)
        yaw   = np.arctan2(R[1, 0], R[0, 0])
    else:
        roll  = np.arctan2(-R[1, 2], R[1, 1])
        pitch = np.arctan2(-R[2, 0], sy)
        yaw   = 0
    return np.array([roll, pitch, yaw])


def origin_to_transform(origin_element):
    """Converts an URDF origin element to a 4x4 transformation matrix."""
    T = np.eye(4)
    if origin_element is not None:
        # Translation
        if "xyz" in origin_element.attrib:
            xyz = np.array([float(v) for v in origin_element.get("xyz").split()])
            T[:3, 3] = xyz
        # Rotation (assumed in RPY)
        if "rpy" in origin_element.attrib:
            rpy = np.array([float(v) for v in origin_element.get("rpy").split()])
            R = euler_to_rotation_matrix(rpy)
            T[:3, :3] = R
    return T


def parse_urdf(file_path):
    """Parses a URDF file and returns a list of boxes with combined transformations.

    For each visual element, the final transform is:

        final_transform = link_transform * visual_origin_transform

    Here the link_transform is computed by composing joint transformations.
    """
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Step 1: Gather visual data per link.
    # This dictionary maps link_name to a list of tuples (visual_origin_transform, size)
    link_visuals = {}
    for link in root.findall("link"):
        link_name = link.get("name")
        for visual in link.findall("visual"):
            origin = visual.find("origin")
            visual_transform = origin_to_transform(origin) if origin is not None else np.eye(4)
            geometry = visual.find("geometry")
            if geometry is None:
                continue
            box_elem = geometry.find("box")
            if box_elem is None:
                continue

            size = np.array([float(v) for v in box_elem.get("size").split()])

            if link_name not in link_visuals:
                link_visuals[link_name] = []
            link_visuals[link_name].append((visual_transform, size))

    # Step 2: Compute the transformation for each link by processing joints.
    # We create a dictionary: link_name -> absolute link transformation (4x4)
    link_transforms = {}

    # First, determine the base links (which are not children of any joint):
    joint_children = set()
    for joint in root.findall("joint"):
        child_elem = joint.find("child")
        if child_elem is not None:
            joint_children.add(child_elem.get("link"))

    for link in root.findall("link"):
        link_name = link.get("name")
        # Base links get the identity transform.
        if link_name not in joint_children:
            link_transforms[link_name] = np.eye(4)

    # Process joints (Note: for more complex URDFs, you might need recursion/traversal)
    for joint in root.findall("joint"):
        parent_elem = joint.find("parent")
        child_elem = joint.find("child")
        if parent_elem is None or child_elem is None:
            continue
        parent_link = parent_elem.get("link")
        child_link = child_elem.get("link")

        joint_origin = joint.find("origin")
        joint_transform = origin_to_transform(joint_origin) if joint_origin is not None else np.eye(4)

        # Compose to get child's absolute transform.
        parent_transform = link_transforms.get(parent_link, np.eye(4))
        link_transforms[child_link] = parent_transform @ joint_transform

    # Step 3: Combine link transform with visual transform
    boxes = []
    for link_name, visuals in link_visuals.items():
        # Get the absolute transform of the link, if not defined assume identity.
        base_T = link_transforms.get(link_name, np.eye(4))
        for visual_transform, size in visuals:
            # The final transformation applied to the box
            total_transform = base_T @ visual_transform
            xyz = total_transform[:3, 3]
            rpy = rotation_matrix_to_euler(total_transform[:3, :3])
            boxes.append((xyz, rpy, size))

    return boxes


def create_box(size, xyz, rpy):
    """Creates an Open3D box mesh with the given size and applies the transformation.

    The box is first created with its center at the origin, then the transformation
    (rotation from RPY and translation) is applied.
    """
    box = o3d.geometry.TriangleMesh.create_box(width=size[0], height=size[1], depth=size[2])
    # Move box center to origin
    box.translate([-size[0] / 2, -size[1] / 2, -size[2] / 2])
    box.compute_vertex_normals()

    # Build transformation matrix from xyz and rpy.
    transform = np.eye(4)
    transform[:3, :3] = euler_to_rotation_matrix(rpy)
    transform[:3, 3] = xyz
    box.transform(transform)

    return box


urdf_file = "data_rlbench/urdf/close_box/box_base.urdf"  # Change to your URDF file path
boxes = parse_urdf(urdf_file)
o3d_boxes = [create_box(size, xyz, rpy) for xyz, rpy, size in boxes]

for box in o3d_boxes:
    box.paint_uniform_color([0.7, 0.7, 0.7])

mesh_coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2)
o3d.visualization.draw_geometries(o3d_boxes + [mesh_coord])

In [117]:
np.asarray(o3d_boxes[0].vertices)

array([[-0.07476484, -0.12500103, -0.17000024],
       [-0.07476492,  0.12499897, -0.17000016],
       [-0.07476508, -0.12500102, -0.18000024],
       [-0.07476516,  0.12499898, -0.18000016],
       [ 0.07523516, -0.12500098, -0.17000384],
       [ 0.07523508,  0.12499902, -0.17000376],
       [ 0.07523492, -0.12500097, -0.18000384],
       [ 0.07523484,  0.12499903, -0.18000376]])

In [4]:
import yaml

mesh = o3d_boxes[0]
yaml_path = f'data_rlbench/urdf/close_box/box_base_0.yaml'

with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

# Process vertices: assuming each three consecutive numbers form a vertex
vertices = np.array(data["vertices"]).reshape(-1, 3)

# Process triangle indices: each three numbers form one face
indices = np.array(data["indices"]).reshape(-1, 3)

# Process texture coordinates (UVs): each pair of numbers forms one uv coordinate.
# IMPORTANT: Open3D expects UVs for each triangle vertex, so the order of values in the yaml
# should correspond to the triangle ordering.
uvs = np.array(data["texture_coordinates"]).reshape(-1, 2)

# Create the triangle mesh and assign vertices and triangles
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(vertices)
mesh.triangles = o3d.utility.Vector3iVector(indices)

# Assign the triangle_uvs.
# Make sure the number of uv coordinates matches the expected number (3 per triangle).
expected_uv_count = len(indices) * 3 // 3  # simply the number of triangles * 3
if len(uvs) != len(indices) * 1:  # if uvs provided are per triangle vertex, then len(uvs)==num_triangles*3
    print("Warning: The number of texture coordinates does not match the expected count per triangle.")
mesh.triangle_uvs = o3d.utility.Vector2dVector(uvs)

# Load texture image using the provided texture savepath
texture_path = data["texture_savepath"]
texture_image = o3d.io.read_image(texture_path)

# Open3D allows a mesh to hold one or more textures. Attach the texture as a list.
mesh.textures = [texture_image]

# (Optional) You can compute vertex normals for proper lighting
mesh.compute_vertex_normals()

# Visualize the mesh with texture
# o3d.visualization.draw_geometries([mesh])
save_path = f'tmp.obj'
o3d.io.write_triangle_mesh(save_path, mesh)
ms = pymeshlab.MeshSet()
ms.load_new_mesh(save_path)
ms.save_current_mesh(save_path)

[Open3D WARNING] Write OBJ can not include triangle normals.


In [5]:
mesh = o3d.io.read_triangle_mesh(save_path)
o3d.visualization.draw_geometries([mesh])